# Understanding Rent in NYC



## Project Setup

In [1]:
# all import statements needed for the project, for example:
import math
import os

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import geopandas as gpd

In [2]:
#Historical monthly average rents by zip code from Zillow from

#create data directory
os.makedirs('data', exist_ok=True)
def download_file(url, directory, filename):
    # Create the directory if it doesn't exist
    os.makedirs(directory, exist_ok=True)

    # Download the file
    response = requests.get(url)
    with open(os.path.join(directory, filename), "wb") as file:
        file.write(response.content)

# Example usage
url = "https://drive.google.com/uc?id=1LdJkhLKTbSfVLSbiZA-S4CF6tLIWKkPj&export=download"
directory = "data"
filename = "zillow_rent_data.csv"

download_file(url, directory, filename)
 


In [3]:

#Historical data from NYC Open Data-trees
TREE_URL = "https://data.cityofnewyork.us/resource/5rq2-4hqu.geojson?$$app_token=bGN5ZHzRAwz2VxsMyRVEiycko"

# Send a GET request to the URL
params = {
    "$where": "latitude IS NOT NULL AND longitude IS NOT NULL AND spc_common IS NOT NULL",
    "$limit": 50  # Set a limit to ensure all data within the date range is fetched
}


response = requests.get(TREE_URL,params)


# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Save the content of the response to a file
    with open("tree_census.geojson", "wb") as f:
        f.write(response.content)
    print("Data downloaded successfully.")
else:
    print("Failed to download data. Status code:", response.status_code)
    


Data downloaded successfully.


In [4]:
## Historical data from NYC Open Data-311complaints
# Define the URL of the dataset
url = "https://data.cityofnewyork.us/resource/erm2-nwe9.geojson?$$app_token=bGN5ZHzRAwz2VxsMyRVEiycko"
# Send a GET request to the URL

params = {
    "$where": "Latitude IS NOT NULL AND created_date >= '2023-02-28T00:00:00.000'",
    "$limit": 50  # Set a limit to ensure all data within the date range is fetched
}

response = requests.get(url,params)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Save the content of the response to a file
    with open("311_complaints.geojson", "wb") as f:
        f.write(response.content)
    print("Data downloaded successfully.")
else:
    print("Failed to download data. Status code:", response.status_code)

Data downloaded successfully.


In [5]:
DB_NAME = "postgres"
DB_USER = "postgres"
DATABASE_URL = f\"postgres+psycopg2://{DB_USER}@localhost/{DB_NAME}\"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"
# Make sure the QUERY_DIRECTORY exists
try:
    os.makedirs(QUERY_DIRECTORY, exist_ok=True)
    print(f"Directory '{QUERY_DIRECTORY}' created successfully or already exists.")
except Exception as e:
    print("Error creating directory:", e)

Directory 'queries' created successfully or already exists.


## Part 1: Data Preprocessing

### Load NYC Open Data on 311 complaints

In [6]:
def load_311data(geojson_file):
    try:
        # Load the GeoJSON file into a GeoDataFrame
        gdf = gpd.read_file(geojson_file)
        print("311 complaints data loaded successfully.")
        return gdf
    except Exception as e:
        print("Error loading 311 complaints data:", str(e))
        return None

# Example usage:
geojson_file = "311_complaints.geojson"
complaints_data = load_311data(geojson_file)

# Step 2: Data Inspection
print(complaints_data.head())

311 complaints data loaded successfully.
  location_state facility_type intersection_street_2              city  \
0                         None       NEW YORK AVENUE          BROOKLYN   
1                         None    ST NICHOLAS AVENUE          NEW YORK   
2                         None       APPLEGATE COURT          BROOKLYN   
3                         None              2 STREET  LONG ISLAND CITY   
4                         None        CONVENT AVENUE          NEW YORK   

  location_zip park_borough            latitude road_ramp        created_date  \
0                  BROOKLYN   40.67244933294371      None 2024-04-13 01:50:54   
1                 MANHATTAN   40.84376204980533      None 2024-04-13 01:50:42   
2                  BROOKLYN  40.600948272530495      None 2024-04-13 01:50:32   
3                    QUEENS   40.74229141366695      None 2024-04-13 01:49:52   
4                 MANHATTAN  40.812364611331716      None 2024-04-13 01:49:48   

  agency  ... resolution_de

In [7]:
### Process NYC Open Data on 311 complaints

In [7]:
def keep_columns(df, columns_to_keep):
    # Select only the specified columns
    df = df[columns_to_keep]
    return df

complaints_data_cleaned = keep_columns(complaints_data, ['unique_key','incident_zip','created_date','complaint_type'])
print(complaints_data_cleaned.head())
print(complaints_data_cleaned.info())

  unique_key incident_zip        created_date           complaint_type
0   60843224        11216 2024-04-13 01:50:54  Noise - Street/Sidewalk
1   60849143        10032 2024-04-13 01:50:42      Noise - Residential
2   60842145        11223 2024-04-13 01:50:32          Illegal Parking
3   60849022        11101 2024-04-13 01:49:52       Indoor Air Quality
4   60842207        10027 2024-04-13 01:49:48  Noise - Street/Sidewalk
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   unique_key      50 non-null     object        
 1   incident_zip    49 non-null     object        
 2   created_date    50 non-null     datetime64[ns]
 3   complaint_type  50 non-null     object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.7+ KB
None


In [9]:
os.getcwd()

'C:\\Users\\wu082\\PycharmProjects\\4501'

### Process Zipcode Data

In [13]:
def load_Zipcode_data(shapefile_dir):
    # Path to the directory containing the shapefile
# Load the shapefile into a GeoDataFrame
    gdf = gpd.read_file(shapefile_dir)
    return gdf
zipcodes=load_Zipcode_data("data/nyc_zipcodes.shp")
print(zipcodes.info())

def keep_Zipcode_columns(df,columns_to_keep):
    df=df[columns_to_keep]
    
    return df

zipcodes_data_cleaned=keep_Zipcode_columns(zipcodes,["ZIPCODE","geometry"])
print(zipcodes_data_cleaned.head())
print(zipcodes_data_cleaned.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ZIPCODE     263 non-null    object  
 1   BLDGZIP     263 non-null    object  
 2   PO_NAME     263 non-null    object  
 3   POPULATION  263 non-null    float64 
 4   AREA        263 non-null    float64 
 5   STATE       263 non-null    object  
 6   COUNTY      263 non-null    object  
 7   ST_FIPS     263 non-null    object  
 8   CTY_FIPS    263 non-null    object  
 9   URL         263 non-null    object  
 10  SHAPE_AREA  263 non-null    float64 
 11  SHAPE_LEN   263 non-null    float64 
 12  geometry    263 non-null    geometry
dtypes: float64(4), geometry(1), object(8)
memory usage: 26.8+ KB
None
  ZIPCODE                                           geometry
0   11436  POLYGON ((1038098.252 188138.380, 1038141.936 ...
1   11213  POLYGON ((1001613.713 186926.440, 1002314.243 ...
2  

### Processing Zillow Rent Data

In [23]:
def load_Zillow_Rent_data(csv_file):
    df = pd.read_csv(csv_file)
    df=df.dropna()
    return df
rent_data=load_Zillow_Rent_data("zillow_rent_data.csv")
print(rent_data.describe())
print(rent_data.head())
print(rent_data.info())

def drop_not_needed_columns(df,columns):
    df = df.drop(columns=columns, errors='ignore')
    return df
columns_to_drop = ["SizeRank", "RegionType", "StateName","State","City","Metro","CountyName"]
rent_data_processed = drop_not_needed_columns(rent_data, columns_to_drop)

def drop_columns_before_date(df, date):
    # Get the index of the date in the columns
    date_index = df.columns.get_loc(date)
    
    # Select columns from the date onwards
    columns_to_keep = df.columns[date_index:]
    
    return df[columns_to_keep]


rent_data_cleaned=pd.merge(drop_columns_before_date(rent_data_processed,"2022-02-28"),rent_data_processed["RegionName"],left_index=True, right_index=True)
rent_data_cleaned=pd.merge(rent_data_cleaned,rent_data_processed["RegionID"],left_index=True, right_index=True)

            RegionID      SizeRank    RegionName   2015-01-31   2015-02-28  \
count     575.000000    575.000000    575.000000   575.000000   575.000000   
mean    87999.222609   1965.786087  52016.845217  1403.794237  1412.462501   
std     51370.757623   1901.496671  30635.660443   628.477602   635.431388   
min     58627.000000      1.000000   2113.000000   551.829341   549.959262   
25%     70823.500000    530.000000  30033.000000  1000.788457  1003.254599   
50%     73201.000000   1339.000000  34698.000000  1229.841194  1231.005300   
75%     94842.500000   2794.500000  85208.500000  1608.809833  1610.500914   
max    417444.000000  12215.000000  98503.000000  5342.134698  5545.637773   

        2015-03-31   2015-04-30   2015-05-31   2015-06-30   2015-07-31  ...  \
count   575.000000   575.000000   575.000000   575.000000   575.000000  ...   
mean   1422.872980  1434.340943  1444.530315  1454.691224  1463.191322  ...   
std     643.096303   650.660652   655.669486   662.678389   

In [24]:
print(rent_data_cleaned.head())
print(rent_data_cleaned.info())

     2022-02-28   2022-03-31   2022-04-30   2022-05-31   2022-06-30  \
0   1762.262439  1770.552861  1785.325091  1810.321876  1833.942229   
2   1721.397080  1742.841518  1764.553406  1744.354038  1743.677698   
8   1838.826094  1835.449047  1856.922199  1879.630889  1890.945343   
16  2415.357857  2451.406046  2492.544828  2528.553196  2585.077670   
18  1871.896033  1909.316349  1903.789149  1937.572633  1945.348547   

     2022-07-31   2022-08-31   2022-09-30   2022-10-31   2022-11-30  ...  \
0   1856.062108  1857.089958  1850.251951  1840.882840  1826.146229  ...   
2   1781.779706  1845.912781  1847.882750  1827.755137  1788.139921  ...   
8   1910.784194  1922.780911  1907.362558  1893.717749  1894.501883  ...   
16  2611.357874  2620.821184  2617.559725  2628.648650  2626.115078  ...   
18  1978.766635  1980.598434  1979.695761  1983.337417  1957.606126  ...   

     2023-06-30   2023-07-31   2023-08-31   2023-09-30   2023-10-31  \
0   1860.805060  1873.335787  1879.080480  18

### Processing NYC Open Data on Trees

In [25]:
def load_trees_data(geojson_file):
    try:
        trees_data = gpd.read_file(geojson_file)
        return trees_data
    except Exception as e:
        print(f"Error loading Trees data: {e}")
        return None

# Example usage:
geojson_file = "tree_census.geojson"
trees_data = load_trees_data(geojson_file)

# Step 2: Data Inspection
if trees_data is not None:
    print(trees_data.head())
    print(trees_data.info())
else:
    print("Tree data not loaded successfully.")

    nta health zipcode     latitude            nta_name     state trnk_wire  \
0  QN17   Fair   11375  40.72309177        Forest Hills  New York        No   
1  QN49   Fair   11357  40.79411067          Whitestone  New York        No   
2  BK90   Good   11211  40.71758074   East Williamsburg  New York        No   
3  BK90   Good   11211  40.71353749   East Williamsburg  New York        No   
4  BK37   Good   11215  40.66677776  Park Slope-Gowanus  New York        No   

            y_sp brnch_othe root_grate  ...            address  sidewalk  \
0  202756.768749         No         No  ...  108-005 70 AVENUE  NoDamage   
1  228644.837379         No         No  ...   147-074 7 AVENUE    Damage   
2  200716.891267         No         No  ...  390 MORGAN AVENUE    Damage   
3  199244.253136         No         No  ...  1027 GRAND STREET    Damage   
4  182202.425999         No         No  ...       603 6 STREET    Damage   

  root_other  created_at borocode block_id trnk_light tree_dbh root_

In [26]:
trees_data_cleaned = keep_columns(trees_data, ['tree_id', 'health','zipcode','status','geometry','spc_latin'])
print(trees_data_cleaned.head())
print(trees_data_cleaned.info())

  tree_id health zipcode status                    geometry  \
0  180683   Fair   11375  Alive  POINT (-73.84422 40.72309)   
1  200540   Fair   11357  Alive  POINT (-73.81868 40.79411)   
2  204026   Good   11211  Alive  POINT (-73.93661 40.71758)   
3  204337   Good   11211  Alive  POINT (-73.93446 40.71354)   
4  189565   Good   11215  Alive  POINT (-73.97598 40.66678)   

                            spc_latin  
0                         Acer rubrum  
1                   Quercus palustris  
2  Gleditsia triacanthos var. inermis  
3  Gleditsia triacanthos var. inermis  
4                     Tilia americana  
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   tree_id    50 non-null     object  
 1   health     50 non-null     object  
 2   zipcode    50 non-null     object  
 3   status     50 non-null     object  
 4   geometry   50 non-nu

## Storing Data

In [27]:
!createdb Project_Database
!psql --dbname Project_Database -c 'CREATE EXTENSION postgis;'


'createdb' is not recognized as an internal or external command,
operable program or batch file.
'psql' is not recognized as an internal or external command,
operable program or batch file.


In [17]:
!pip install psycopg2-binary

In [30]:
import psycopg2

# dbname = "Project_Database"
# user = "sfm"
# host = "localhost"
# port = "5432"

dbname = "postgres"
user = "postgres"
host = "localhost"
password = "postgres"
port = "5432"

try:
    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    print("Connection successful")

    # Do some operations with the database here

    # Close the connection
    conn.close()
    print("Connection closed")
except psycopg2.Error as e:
    print("Unable to connect to the database:", e)

Connection successful
Connection closed


In [45]:

engine = db.create_engine(DATABASE_URL)

In [32]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
NYC_zip_codes_SCHEMA = """
CREATE TABLE IF NOT EXISTS Zip_Code
(
    id SERIAL PRIMARY KEY,
    zipcode INTEGER,
    geometry GEOMETRY(POINT, 4326)
);
"""

ThreeOneOne_complaints_SCHEMA = """
CREATE TABLE IF NOT EXISTS Complaint
(
    unique_key SERIAL PRIMARY KEY,
    incident_zip INTEGER,
    created_date TIMESTAMP,
    complaint_type TEXT
);
"""

Historical_Average_Rents_SCHEMA = """
CREATE TABLE IF NOT EXISTS Rent
(
    id SERIAL PRIMARY KEY,
    "2022-02-28" FLOAT,
    "2022-03-31" FLOAT,
    "2022-04-30" FLOAT,
    "2022-05-31" FLOAT,
    "2022-06-30" FLOAT,
    "2022-07-31" FLOAT,
    "2022-08-31" FLOAT,
    "2022-09-30" FLOAT,
    "2022-10-31" FLOAT,
    "2022-11-30" FLOAT,
    "2022-12-31" FLOAT,
    "2023-01-31" FLOAT,
    "2023-02-28" FLOAT,
    "2023-03-31" FLOAT,
    "2023-04-30" FLOAT,
    "2023-05-31" FLOAT,
    "2023-06-30" FLOAT,
    "2023-07-31" FLOAT,
    "2023-08-31" FLOAT,
    "2023-09-30" FLOAT,
    "2023-10-31" FLOAT,
    "2023-11-30" FLOAT,
    "2023-12-31" FLOAT,
    "2024-01-31" FLOAT,
    RegionName INT,
    RegionID INT
);
"""
Trees_SCHEMA = """
CREATE TABLE IF NOT EXISTS Tree
(
    tree_id TEXT PRIMARY KEY,
    health TEXT,
    zipcode TEXT,
    status TEXT,
    geometry GEOMETRY(Point, 4326),
    spc_latin TEXT
);
"""


In [33]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(NYC_zip_codes_SCHEMA)
    f.write("\n")
    f.write(ThreeOneOne_complaints_SCHEMA)
    f.write("\n")
    f.write(Historical_Average_Rents_SCHEMA)
    f.write("\n")
    f.write(Trees_SCHEMA)

In [38]:
# Connect to your PostgreSQL database
conn = psycopg2.connect(
    # dbname="Project_Database",
    # user="sfm",
    # host="localhost",
    # port="5432"
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port

)

# Open a cursor to perform database operations
with conn.cursor() as cur:
    # Read and execute the schema files
    with open(DATABASE_SCHEMA_FILE, 'r') as f:
        sql_commands = f.read().split(';')
        for command in sql_commands:
            if command.strip() != '':
                cur.execute(command)


# Commit the changes
conn.commit()

# Close the connection
conn.close()

### Add Data to Database

In [54]:
import psycopg2.extras as extras
def execute_values(conn, df, table): 
  
    tuples = [tuple(x) for x in df.to_numpy()] 
  
    cols = ','.join(list(df.columns)) 
    # SQL query to execute 
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols) 
    cursor = conn.cursor() 
    try: 
        extras.execute_values(cursor, query, tuples) 
        conn.commit() 
    except (Exception, psycopg2.DatabaseError) as error: 
        print(f"Error inserting {table}: {error}") 
        conn.rollback() 
        cursor.close() 
        return 1
    print(f"the dataframe: {table} is inserted") 
    cursor.close() 

def write_dataframes_to_table(table_to_df_dict,table_name):
    df = table_to_df_dict[table_name]
    conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port
    )
    execute_values(conn, df, table_name) 
    

In [55]:
map_table_name_to_dataframe = {
    "Zip_Code": zipcodes_data_cleaned,
    "Complaint": complaints_data_cleaned,
    "Rent": rent_data_cleaned,
    "Tree": trees_data_cleaned,
}

In [56]:
write_dataframes_to_table(map_table_name_to_dataframe,'Zip_Code')
write_dataframes_to_table(map_table_name_to_dataframe,'Complaint')
write_dataframes_to_table(map_table_name_to_dataframe,'Rent')
write_dataframes_to_table(map_table_name_to_dataframe,'Tree')

Error inserting Zip_Code: can't adapt type 'Polygon'
Error inserting Complaint: duplicate key value violates unique constraint "complaint_pkey"
DETAIL:  Key (unique_key)=(60843224) already exists.

Error inserting Rent: syntax error at or near "2022"
LINE 1: INSERT INTO Rent(2022-02-28,2022-03-31,2022-04-30,2022-05-31...
                         ^

Error inserting Tree: can't adapt type 'Point'


## Part 3: Understanding the Data

In [46]:
# Helper function to write the queries to file
QUERY_DIRECTORY = "queries"

def write_query_to_file(query, filename):
    filepath = os.path.join(QUERY_DIRECTORY, filename)
    with open(filepath, 'w') as file:
        file.write(query)

### Query 1

In [49]:
db.__version__

'2.0.29'

In [47]:
QUERY_1_FILENAME = "complaints_by_zipcode.sql"

QUERY_1 = """
SELECT count(id) As "Total Number of Complaints", IncidentZip AS "ZipCode" FROM complaints
Where CAST(CreatedDate AS DATE)>="2023-03-01" and CAST(CreatedDate AS DATE)<='2024-02-29'
GROUP BY IncidentZip
ORDER BY count(id) desc
"""

In [50]:
# engine.execute(QUERY_1).fetchall()
from sqlalchemy import text

with engine.connect() as connection:
    result = connection.execute(text(QUERY_1))
    print(result)


OperationalError: (sqlite3.OperationalError) no such table: complaints
[SQL: 
SELECT count(id) As "Total Number of Complaints", IncidentZip AS "ZipCode" FROM complaints
Where CAST(CreatedDate AS DATE)>="2023-03-01" and CAST(CreatedDate AS DATE)<='2024-02-29'
GROUP BY IncidentZip
ORDER BY count(id) desc
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

### Query 2

In [ ]:
QUERY_2_FILENAME = "top_10_zipcodes_by_trees.sql"

QUERY_2 = """
SELECT count(tree_id) As "Total Number of Trees", zipcode AS "ZipCode" FROM trees GROUP BY zipcode
ORDER BY count(tree_id) desc LIMIT 10
"""

In [ ]:
engine.execute(QUERY_2).fetchall()

In [ ]:
write_query_to_file(QUERY_2, QUERY_2_FILENAME)

### Query 3

In [ ]:
QUERY_3_FILENAME = "average_rent_in_top_tree_zipcodes.sql"

QUERY_3 = """
SELECT count(t.tree_id) As "Total Number of Trees", round(AVG(r.2024-01-31),2) AS "Rent" FROM trees t JOIN
rent r on t.zipcode=r.RegionName
Where t.zipcode in (SELECT zipcode FROM trees GROUP BY zipcode
ORDER BY count(tree_id) desc LIMIT 10)
GROUP BY t.zipcode
ORDER BY count(t.tree_id)desc
"""

In [ ]:
engine.execute(QUERY_3).fetchall()

In [ ]:
write_query_to_file(QUERY_3, QUERY_3_FILENAME)

### Query 4

In [ ]:
QUERY_4_FILENAME = "rent_tree_complaint_correlation.sql"

QUERY_4 = """
(SELECT c.IncidentZip AS "ZipCode", round(AVG(r.2024-01-31),2) AS "Average Rent",count(t.tree_id) AS "Total Number of Trees", count(c.id) As "Total Number of Complaints" FROM complaints c
RIGHTJOIN rent r on c.IncidentZip=r.RegionName LEFT JOIN trees t on t.zipcode=r.RegionName
GROUP BY c.IncidentZip
ORDER BY round(AVG(r.2024-01-31),2) desc LIMIT 5)
UNION
(SELECT c.IncidentZip AS "ZipCode", round(AVG(r.2024-01-31),2) AS "Average Rent",COUNT(COALESCE(t.tree_id, 0)) AS "Total Number of Trees", COUNT(COALESCE(c.id, 0)) As "Total Number of Complaints" FROM complaints c
RIGHT JOIN rent r on c.IncidentZip=r.RegionName LEFT JOIN trees t on t.zipcode=r.RegionName
GROUP BY c.IncidentZip
ORDER BY round(AVG(r.2024-01-31),2) asc LIMIT 5)
"""

In [ ]:
engine.execute(QUERY_4).fetchall()

In [ ]:
write_query_to_file(QUERY_4, QUERY_4_FILENAME)

### Query 5

In [ ]:
QUERY_5_FILENAME = "most_greenery_join.sql"

QUERY_5 = """
SELECT z.zipcode, COUNT(t.tree_id) AS total_trees
FROM trees t
JOIN zipcodes z ON ST_Contains(z.geom, ST_SetSRID(ST_MakePoint(t.longitude, t.Latitude), 4326))
GROUP BY z.zipcode
ORDER BY total_trees DESC
LIMIT 10;
"""

In [ ]:
engine.execute(QUERY_5).fetchall()

In [ ]:
write_query_to_file(QUERY_5, QUERY_5_FILENAME)

### Query 6

In [ ]:
pip install shapely
from shapely.geometry import Point
import pyproj

# Define the point and buffer radius in meters
latitude, longitude = 40.80737875669467, -73.96253174434912
buffer_radius_miles = 0.5

# Convert buffer radius from miles to meters
buffer_radius_meters = buffer_radius_miles * 1609.34

# Define the coordinate reference system (CRS) for WGS 84 (EPSG:4326)
crs_wgs84 = pyproj.CRS("EPSG:4326")

# Create a point geometry
point = Point(longitude, latitude)

# Project the point to a local CRS in meters (e.g., EPSG:3857)
crs_local = pyproj.CRS("EPSG:3857")
projected_point = pyproj.Transformer.from_crs(crs_wgs84, crs_local, always_xy=True).transform(point.x, point.y)

# Create a buffer around the point
buffered_point = Point(projected_point).buffer(buffer_radius_meters)

# Convert the buffer to WKT format
buffer_wkt = buffered_point.wkt

QUERY_6_FILENAME = "trees_within_half_mile_radius.sql"

QUERY_6 = """
SELECT
    tree_id, spc_latin, health, status, ST_AsText(the_geom) AS location
FROM
    trees
WHERE
    ST_DWithin(the_geom::geography, ST_SetSRID(ST_GeomFromText('POINT(-73.96253174434912 40.80737875669467)'), 4326)::geography, 804.67) = TRUE
    """


In [ ]:
engine.execute(QUERY_6).fetchall()

In [ ]:
write_query_to_file(QUERY_6, QUERY_6_FILENAME)

In [ ]:
psql --dbname groupNproject -f query.sql

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
def get_data_for_visual_1():
    conn = sqlite3.connect('Project_Database.db')
    
    # Define your SQL query
    sql_query = """
    SELECT count(id) As "Total Number of Complaints", complaint_type"Type" FROM complaints
    Where CAST(created_date AS DATE)>="2023-03-01" and CAST(created_date AS DATE)<='2024-02-29'
    GROUP BY complaint_type
    ORDER BY count(id) desc
    LIMIT 3
    """
    
    # Execute the query and fetch the results into a DataFrame
    data = pd.read_sql_query(sql_query, conn)
    
    # Close the database connection
    conn.close()
    
    return data


In [ ]:
def plot_visual_1(dataframe):
    conn = sqlite3.connect('Project_Database.db')
    dates = pd.date_range(start='2023-03-01', end='2024-02-29')
    data = []
    for complaint_type in dataframe['complaint_type']:
        sql_query = f"""
        SELECT created_date, COUNT(id) AS daily_complaints
        FROM complaints
        WHERE created_date BETWEEN '2023-03-01' AND '2024-02-29'
        AND complaint_type = '{complaint_type}'
        GROUP BY created_date
        """
        df = pd.read_sql_query(sql_query, conn)
        df.set_index('created_date', inplace=True)
        data.append(df.reindex(dates).fillna(0)['daily_complaints'])
    
    # Calculate the total number of complaints for the top 3 complaint types for each day
    total_complaints = sum(data)
    
    conn.close()
    
    plt.figure(figsize=(15, 8))
    plt.plot(dates, total_complaints, label='Total Complaints', color='black', linewidth=2, linestyle='--')
    plt.title('Number of Complaints per Day for Top 3 Complaint Types')
    plt.xlabel('Date')
    plt.ylabel('Number of Complaints')
    plt.grid(True)
    plt.legend(title='Complaint Type', loc='upper right')
    plt.show()

In [ ]:
visual1_dataframe = get_data_for_visual_1()
plot_visual_1(visual1_dataframe)

### Visualization 2

In [ ]:
def get_data_for_visual_2():
    conn = sqlite3.connect('Project_Database.db')
    
    # Define your SQL query
    sql_query = """
    SELECT count(id) As "Total Number of Complaints", complaint_type"Type" FROM complaints
    Where CAST(created_date AS DATE)>="2023-03-01" and CAST(created_date AS DATE)<='2024-02-29' and IncidentZip=10027
    GROUP BY complaint_type
    ORDER BY count(id) desc
    LIMIT 10
    """
    
    # Execute the query and fetch the results into a DataFrame
    data = pd.read_sql_query(sql_query, conn)
    
    # Close the database connection
    conn.close()
    
    return data


In [ ]:
def plot_visual_2(visual2_dataframe):
    plt.figure(figsize=(12, 6))
    sns.barplot(x='Total Number of Complaints', y='Type', data=visual2_dataframe, palette='viridis')
    plt.title('Top 10 Complaint Types in Zip Code 10027 (Mar 1, 2022 - Feb 29, 2024)')
    plt.xlabel('Total Number of Complaints')
    plt.ylabel('Complaint Type')
    plt.show()

In [ ]:
visual2_dataframe = get_data_for_visual_2()
plot_visual_2(visual2_dataframe)

### Visualization 3 (in progress)

In [ ]:
def get_data_for_visual_2():
    conn = sqlite3.connect('Project_Database.db')
    
    # Define your SQL query
    sql_query = """
    SELECT count(id) As "Total Number of Complaints", complaint_type"Type" FROM complaints
    Where CAST(created_date AS DATE)>="2023-03-01" and CAST(created_date AS DATE)<='2024-02-29' and IncidentZip=10027
    
    GROUP BY complaint_type
    ORDER BY count(id) desc
    LIMIT 10
    """
    
    # Execute the query and fetch the results into a DataFrame
    data = pd.read_sql_query(sql_query, conn)
    
    # Close the database connection
    conn.close()
    
    return data


In [ ]:
visual2_dataframe = get_data_for_visual_2()
plot_visual_2(visual2_dataframe)

### Visualization 4

### Visualization 5

### Visualization 6